## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 4 Aug. 10


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,August-4-2025,Condo/Co-op,7635 SW 57th Ave #2,South Miami,FL,33143.0,370000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/South-Miami/7635-SW-...,MARMLS,A11823219,N,Y,25.699997,-80.28494


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-08-05    28
2025-08-08    25
2025-08-06    17
2025-08-07    10
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
29,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036
78,PAST SALE,2025-08-07,Condo/Co-op,7255 Fisher Island Dr #7255,Miami Beach,FL,33109.0,9000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,MARMLS,A11737939,N,Y,25.762287,-80.134708
20,PAST SALE,2025-08-05,Condo/Co-op,500 Alton Rd #3703,Miami Beach,FL,33139.0,6700000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11820816,N,Y,25.775127,-80.141257
65,PAST SALE,2025-08-05,Condo/Co-op,10225 Collins Ave #901,Bal Harbour,FL,33154.0,4700000.0,3.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,MARMLS,A11767389,N,Y,25.896548,-80.123085
45,PAST SALE,2025-08-07,Condo/Co-op,5000 Island Estates Dr #1208,Aventura,FL,33160.0,4125000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/5000-Island...,MARMLS,A11785521,N,Y,25.947527,-80.130447
84,PAST SALE,2025-08-06,Condo/Co-op,6899 Collins Ave #2804,Miami Beach,FL,33141.0,3530000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11698436,N,Y,25.853732,-80.120089
36,PAST SALE,2025-08-07,Condo/Co-op,2301 Collins Ave #1407,Miami Beach,FL,33139.0,2400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Beaches MLS,F10506939,N,Y,25.798828,-80.127201
89,PAST SALE,2025-08-05,Condo/Co-op,605 Ocean Dr Unit 4L,Key Biscayne,FL,33149.0,2120000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/605-Oce...,MARMLS,A11701981,N,Y,25.690092,-80.157457
30,PAST SALE,2025-08-05,Condo/Co-op,200 Biscayne Boulevard Way #4702,Miami,FL,33131.0,1750000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11818915,N,Y,25.770430,-80.189163
33,PAST SALE,2025-08-05,Condo/Co-op,288 Sea View Dr Unit C2,Key Biscayne,FL,33149.0,1665000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/288-Sea...,MARMLS,A11805542,N,Y,25.685372,-80.158768


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

80


In [20]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,028,539


In [21]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$563


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$82,283,112


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
97
-----------
Input Previous Week Condo Average Sales Price:
1078191
-----------
Input Previous Week Condo Average PSF:
570
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
104584533.0


In [29]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,97,1078191,570,104584533.0


In [30]:
last_week['Last Week Total Dollar Volume'].iloc[0]

104584533.0

In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           80             1028539          563                     82283112.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [31]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [32]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [33]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [34]:
pd.set_option('display.max_columns',None)

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,JADE SIGNATURE CONDO,5689.0,NaN,2018.0,NaN,2605.0,11378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rejane Gomes de Paula,Fortune Christie's Intl R.E.,None,None,Abe Ash,"Multivest Realty, Inc.",None,None,1,orange


In [36]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [37]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,JADE SIGNATURE CONDO,5689.0,NaN,2018.0,NaN,2605.0,11378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rejane Gomes de Paula,Fortune Christie's Intl R.E.,None,None,Abe Ash,"Multivest Realty, Inc.",None,None,1,orange
1,PAST SALE,2025-08-07,Condo/Co-op,7255 Fisher Island Dr #7255,Miami Beach,FL,33109.0,9000000.0,3.0,3.5,Fisher Island,3140.0,NaN,1992.0,NaN,2866.0,8521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,MARMLS,A11737939,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue
2,PAST SALE,2025-08-05,Condo/Co-op,500 Alton Rd #3703,Miami Beach,FL,33139.0,6700000.0,2.0,2.5,FIVE PARK RESIDENCES,2615.0,NaN,2024.0,NaN,2562.0,6998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11820816,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Nelson Gonzalez,BHHS EWM Realty,None,None,Jordan Karp,Jordan Karp LLC,None,None,3,blue
3,PAST SALE,2025-08-05,Condo/Co-op,10225 Collins Ave #901,Bal Harbour,FL,33154.0,4700000.0,3.0,6.5,BELLINI CONDO,3820.0,NaN,2004.0,NaN,1230.0,5890.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,MARMLS,A11767389,N,Y,25.896548,-80.123085,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,Jill Hertzberg,Coldwell Banker Realty,None,None,Raquel Sragowicz,Citadel Realty,None,None,4,blue
4,PAST SALE,2025-08-07,Condo/Co-op,5000 Island Estates Dr #1208,Aventura,FL,33160.0,4125000.0,4.0,5.5,PRIVE CONDO,4119.0,NaN,2018.0,NaN,1001.0,8440.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/5000-Island...,MARMLS,A11785521,N,Y,25.947527,-80.130447,https://www.redfin.com/FL/Aventura/5000-Island...,Richard Goihman,Douglas Elliman,None,None,Salesperson Non Member,Non-MLS Member,None,None,5,blue
5,PAST SALE,2025-08-06,Condo/Co-op,6899 Collins Ave #2804,Miami Beach,FL,33141.0,3530000.0,3.0,2.5,NORTH CARILLON BEACH COND,2150.0,NaN,2008.0,NaN,1642.0,6298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11698436,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Eileen Nexer,Douglas Elliman,Levente Kovary,Douglas Elliman,Levente Kovary,Douglas Elliman,None,None,6,blue
6,PAST SALE,2025-08-07,Condo/Co-op,2301 Collins Ave #1407,Miami Beach,FL,33139.0,2400000.0,2.0,2.0,Roney Palace,1230.0,NaN,1926.0,NaN,1951.0,2442.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Beaches MLS,F10506939,N,Y,25.798828,-80.127201,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Erin Argiropoulos,RE/MAX Preferred,Rachael Barach,RE/MAX Preferred,Sophia Albrecht,Thine Agency Inc,None,None,7,blue
7,PAST SALE,2025-08-05,Condo/Co-op,605 Ocean Dr Unit 4L,Key Biscayne,FL,33149.0,2120000.0,3.0,3.0,The Sands of Key Biscayne,1772.0,NaN,1969.0,NaN,1196.0,2602.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/605-Oce...,MARMLS,A11701981,N,Y,25.690092,-80.157457,https://www.redfin.com/FL/Key-Biscayne/605-Oce...,Rene Iglesias,One Sotheby's International Realty,None,None,Julian Osuna Ospina,Keller Williams Capital Realty,Danielle Jordan,Keller Williams Capital Realty,8,blue
8,PAST SALE,2025-08-05,Condo/Co-op,200 Biscayne Boulevard Way #4702,Miami,FL,33131.0,1750000.0,3.0,3.5,EPIC WEST CONDO,2457.0,NaN,2008.0,NaN,712.0,3483.0,Sold,NaN,NaN,https://

In [38]:
%store -r map_box_api_key

In [39]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [40]:
m.save('index.html')

## Data snagger

In [41]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [42]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,JADE SIGNATURE CONDO,5689.0,NaN,2018.0,NaN,2605.0,11378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rejane Gomes de Paula,Fortune Christie's Intl R.E.,None,None,Abe Ash,"Multivest Realty, Inc.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_08112025


# CREATE TEMPLATE 

In [44]:
muni_set = set(df_top_ten['CITY'])

In [45]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [46]:
df_top_ten.reset_index(inplace=True,drop=True)

In [47]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [48]:
top_sale

'JADE SIGNATURE CONDO at 16901 Collins Ave #5103 in Sunny Isles Beach'

In [49]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [50]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [51]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,JADE SIGNATURE CONDO,5689.0,NaN,2018.0,NaN,2605.0,11378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rejane Gomes de Paula,Fortune Christie's Intl R.E.,None,None,Abe Ash,"Multivest Realty, Inc.",None,None,1,orange,JADE SIGNATURE CONDO at 16901 Collins Ave #510...
1,PAST SALE,2025-08-07,Condo/Co-op,7255 Fisher Island Dr #7255,Miami Beach,FL,33109.0,9000000.0,3.0,3.5,Fisher Island,3140.0,NaN,1992.0,NaN,2866.0,8521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,MARMLS,A11737939,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue,Fisher Island at 7255 Fisher Island Dr #7255 i...
2,PAST SALE,2025-08-05,Condo/Co-op,500 Alton Rd #3703,Miami Beach,FL,33139.0,6700000.0,2.0,2.5,FIVE PARK RESIDENCES,2615.0,NaN,2024.0,NaN,2562.0,6998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11820816,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Nelson Gonzalez,BHHS EWM Realty,None,None,Jordan Karp,Jordan Karp LLC,None,None,3,blue,FIVE PARK RESIDENCES at 500 Alton Rd #3703 in ...
3,PAST SALE,2025-08-05,Condo/Co-op,10225 Collins Ave #901,Bal Harbour,FL,33154.0,4700000.0,3.0,6.5,BELLINI CONDO,3820.0,NaN,2004.0,NaN,1230.0,5890.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,MARMLS,A11767389,N,Y,25.896548,-80.123085,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,Jill Hertzberg,Coldwell Banker Realty,None,None,Raquel Sragowicz,Citadel Realty,None,None,4,blue,BELLINI CONDO at 10225 Collins Ave #901 in Bal...
4,PAST SALE,2025-08-07,Condo/Co-op,5000 Island Estates Dr #1208,Aventura,FL,33160.0,4125000.0,4.0,5.5,PRIVE CONDO,4119.0,NaN,2018.0,NaN,1001.0,8440.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/5000-Island...,MARMLS,A11785521,N,Y,25.947527,-80.130447,https://www.redfin.com/FL/Aventura/5000-Island...,Richard Goihman,Douglas Elliman,None,None,Salesperson Non Member,Non-MLS Member,None,None,5,blue,PRIVE CONDO at 5000 Island Estates Dr #1208 in...
5,PAST SALE,2025-08-06,Condo/Co-op,6899 Collins Ave #2804,Miami Beach,FL,33141.0,3530000.0,3.0,2.5,NORTH CARILLON BEACH COND,2150.0,NaN,2008.0,NaN,1642.0,6298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11698436,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Eileen Nexer,Douglas Elliman,Levente Kovary,Douglas Elliman,Levente Kovary,Douglas Elliman,None,None,6,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
6,PAST SALE,2025-08-07,Condo/Co-op,2301 Collins Ave #1407,Miami Beach,FL,33139.0,2400000.0,2.0,2.0,Roney Palace,1230.0,NaN,1926.0,NaN,1951.0,2442.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Beaches MLS,F10506939,N,Y,25.798828,-80.127201,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Erin Argiropoulos,RE/MAX Preferred,Rachael Barach,RE/MAX Preferred,Sophia Albrecht,Thine Agency Inc,None,None,7,blue,Roney Palace at 2301 Collins Ave #1407 in Miam...
7,PAST SALE,2025-08-05,Condo/Co-op,605 Ocean Dr Unit 4L,Key Biscayne,FL,33149.0,2120000.0,3.0,3.0,The Sands of Key Biscayne,1772.0,NaN,1969.0,NaN,1196.0,2602.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/605-Oce...,MARMLS,A11701981,N,Y,25.690092,-80.157457,https://www.redfin.com/FL/Key

In [52]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: JADE SIGNATURE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,665,000 to $14,820,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Bal Harbour, Sunny Isles Beach, Key Biscayne, Aventura, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 80 condo sales totaling $82,283,112 million from Aug. 4 to Aug. 10. The previous week, brokers closed 97 condo sales totaling $104,584,533.

Last week’s units sold for an average of $1,028,539, lower than the $1,078,191 

In [53]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [54]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [55]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [56]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,JADE SIGNATURE CONDO,5689.0,NaN,2018.0,NaN,2605.0,11378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rejane Gomes de Paula,Fortune Christie's Intl R.E.,None,None,Abe Ash,"Multivest Realty, Inc.",None,None,1,orange,JADE SIGNATURE CONDO at 16901 Collins Ave #510...
1,PAST SALE,2025-08-07,Condo/Co-op,7255 Fisher Island Dr #7255,Miami Beach,FL,33109.0,9000000.0,3.0,3.5,Fisher Island,3140.0,NaN,1992.0,NaN,2866.0,8521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,MARMLS,A11737939,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue,Fisher Island at 7255 Fisher Island Dr #7255 i...
2,PAST SALE,2025-08-05,Condo/Co-op,500 Alton Rd #3703,Miami Beach,FL,33139.0,6700000.0,2.0,2.5,FIVE PARK RESIDENCES,2615.0,NaN,2024.0,NaN,2562.0,6998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11820816,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Nelson Gonzalez,BHHS EWM Realty,None,None,Jordan Karp,Jordan Karp LLC,None,None,3,blue,FIVE PARK RESIDENCES at 500 Alton Rd #3703 in ...
3,PAST SALE,2025-08-05,Condo/Co-op,10225 Collins Ave #901,Bal Harbour,FL,33154.0,4700000.0,3.0,6.5,BELLINI CONDO,3820.0,NaN,2004.0,NaN,1230.0,5890.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,MARMLS,A11767389,N,Y,25.896548,-80.123085,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,Jill Hertzberg,Coldwell Banker Realty,None,None,Raquel Sragowicz,Citadel Realty,None,None,4,blue,BELLINI CONDO at 10225 Collins Ave #901 in Bal...
4,PAST SALE,2025-08-07,Condo/Co-op,5000 Island Estates Dr #1208,Aventura,FL,33160.0,4125000.0,4.0,5.5,PRIVE CONDO,4119.0,NaN,2018.0,NaN,1001.0,8440.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/5000-Island...,MARMLS,A11785521,N,Y,25.947527,-80.130447,https://www.redfin.com/FL/Aventura/5000-Island...,Richard Goihman,Douglas Elliman,None,None,Salesperson Non Member,Non-MLS Member,None,None,5,blue,PRIVE CONDO at 5000 Island Estates Dr #1208 in...
5,PAST SALE,2025-08-06,Condo/Co-op,6899 Collins Ave #2804,Miami Beach,FL,33141.0,3530000.0,3.0,2.5,NORTH CARILLON BEACH COND,2150.0,NaN,2008.0,NaN,1642.0,6298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11698436,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Eileen Nexer,Douglas Elliman,Levente Kovary,Douglas Elliman,Levente Kovary,Douglas Elliman,None,None,6,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
6,PAST SALE,2025-08-07,Condo/Co-op,2301 Collins Ave #1407,Miami Beach,FL,33139.0,2400000.0,2.0,2.0,Roney Palace,1230.0,NaN,1926.0,NaN,1951.0,2442.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Beaches MLS,F10506939,N,Y,25.798828,-80.127201,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Erin Argiropoulos,RE/MAX Preferred,Rachael Barach,RE/MAX Preferred,Sophia Albrecht,Thine Agency Inc,None,None,7,blue,Roney Palace at 2301 Collins Ave #1407 in Miam...
7,PAST SALE,2025-08-05,Condo/Co-op,605 Ocean Dr Unit 4L,Key Biscayne,FL,33149.0,2120000.0,3.0,3.0,The Sands of Key Biscayne,1772.0,NaN,1969.0,NaN,1196.0,2602.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/605-Oce...,MARMLS,A11701981,N,Y,25.690092,-80.157457,https://www.redfin.com/FL/Key

In [57]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: JADE SIGNATURE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,665,000 to $14,820,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Bal Harbour, Sunny Isles Beach, Key Biscayne, Aventura, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 80 condo sales totaling $82,283,112 million from Aug. 4 to Aug. 10. The previous week, brokers closed 97 condo sales totaling $104,584,533.

Last week’s units sold for an average of $1,028,539, lower than the $1,078,191 

In [58]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-5103/home/56743874


In [59]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/7255-Fisher-Island-Dr-33109/unit-7255/home/42804078


In [60]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Key-Biscayne/288-Sea-View-Dr-33149/unit-C2/home/42924434


In [61]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/200-Biscayne-Boulevard-Way-33131/unit-4702/home/42679797


In [62]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/7255-Fisher-Island-Dr-33109/unit-7255/home/42804078


In [63]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/200-Biscayne-Boulevard-Way-33131/unit-4702/home/42679797


In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-06,Condo/Co-op,16901 Collins Ave #5103,Sunny Isles Beach,FL,33160.0,14820000.0,5.0,6.5,JADE SIGNATURE CONDO,5689.0,NaN,2018.0,NaN,2605.0,11378.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11823229,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Rejane Gomes de Paula,Fortune Christie's Intl R.E.,None,None,Abe Ash,"Multivest Realty, Inc.",None,None,1,orange,JADE SIGNATURE CONDO at 16901 Collins Ave #510...
1,PAST SALE,2025-08-07,Condo/Co-op,7255 Fisher Island Dr #7255,Miami Beach,FL,33109.0,9000000.0,3.0,3.5,Fisher Island,3140.0,NaN,1992.0,NaN,2866.0,8521.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,MARMLS,A11737939,N,Y,25.762287,-80.134708,https://www.redfin.com/FL/Miami-Beach/7255-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue,Fisher Island at 7255 Fisher Island Dr #7255 i...
2,PAST SALE,2025-08-05,Condo/Co-op,500 Alton Rd #3703,Miami Beach,FL,33139.0,6700000.0,2.0,2.5,FIVE PARK RESIDENCES,2615.0,NaN,2024.0,NaN,2562.0,6998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11820816,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Nelson Gonzalez,BHHS EWM Realty,None,None,Jordan Karp,Jordan Karp LLC,None,None,3,blue,FIVE PARK RESIDENCES at 500 Alton Rd #3703 in ...
3,PAST SALE,2025-08-05,Condo/Co-op,10225 Collins Ave #901,Bal Harbour,FL,33154.0,4700000.0,3.0,6.5,BELLINI CONDO,3820.0,NaN,2004.0,NaN,1230.0,5890.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,MARMLS,A11767389,N,Y,25.896548,-80.123085,https://www.redfin.com/FL/Bal-Harbour/10225-Co...,Jill Hertzberg,Coldwell Banker Realty,None,None,Raquel Sragowicz,Citadel Realty,None,None,4,blue,BELLINI CONDO at 10225 Collins Ave #901 in Bal...
4,PAST SALE,2025-08-07,Condo/Co-op,5000 Island Estates Dr #1208,Aventura,FL,33160.0,4125000.0,4.0,5.5,PRIVE CONDO,4119.0,NaN,2018.0,NaN,1001.0,8440.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/5000-Island...,MARMLS,A11785521,N,Y,25.947527,-80.130447,https://www.redfin.com/FL/Aventura/5000-Island...,Richard Goihman,Douglas Elliman,None,None,Salesperson Non Member,Non-MLS Member,None,None,5,blue,PRIVE CONDO at 5000 Island Estates Dr #1208 in...
5,PAST SALE,2025-08-06,Condo/Co-op,6899 Collins Ave #2804,Miami Beach,FL,33141.0,3530000.0,3.0,2.5,NORTH CARILLON BEACH COND,2150.0,NaN,2008.0,NaN,1642.0,6298.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11698436,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Eileen Nexer,Douglas Elliman,Levente Kovary,Douglas Elliman,Levente Kovary,Douglas Elliman,None,None,6,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
6,PAST SALE,2025-08-07,Condo/Co-op,2301 Collins Ave #1407,Miami Beach,FL,33139.0,2400000.0,2.0,2.0,Roney Palace,1230.0,NaN,1926.0,NaN,1951.0,2442.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Beaches MLS,F10506939,N,Y,25.798828,-80.127201,https://www.redfin.com/FL/Miami-Beach/2301-Col...,Erin Argiropoulos,RE/MAX Preferred,Rachael Barach,RE/MAX Preferred,Sophia Albrecht,Thine Agency Inc,None,None,7,blue,Roney Palace at 2301 Collins Ave #1407 in Miam...
7,PAST SALE,2025-08-05,Condo/Co-op,605 Ocean Dr Unit 4L,Key Biscayne,FL,33149.0,2120000.0,3.0,3.0,The Sands of Key Biscayne,1772.0,NaN,1969.0,NaN,1196.0,2602.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/605-Oce...,MARMLS,A11701981,N,Y,25.690092,-80.157457,https://www.redfin.com/FL/Key

## Time on Market Calculator

In [65]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 5, 17) ## List (Earlier) date
date2 = datetime(2025, 8, 16) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

91


## Clean CSV for Datawrapper Chart

In [66]:
chart_df = df_top_ten

In [67]:
chart_df = chart_df.fillna(" ")

In [68]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [69]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [70]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [71]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [72]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [73]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [74]:
chart_df['PPSF'].astype(float)

0    2605.0
1    2866.0
2    2562.0
3    1230.0
4    1001.0
5    1642.0
6    1951.0
7    1196.0
8     712.0
9     925.0
Name: PPSF, dtype: float64

In [75]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [76]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [77]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-08-06,JADE SIGNATURE CONDO,16901 Collins Ave #5103 in Sunny Isles Beach,"$14,820,000","5,689","$2,605",2018,Rejane Gomes de Paula Fortune Christie's Intl ...,"Abe Ash Multivest Realty, Inc."
1,2025-08-07,Fisher Island,7255 Fisher Island Dr #7255 in Miami Beach,"$9,000,000","3,140","$2,866",1992,"Denise Levine Denise LeVine Realty, Inc.",Jill Eber Coldwell Banker Realty
2,2025-08-05,FIVE PARK RESIDENCES,500 Alton Rd #3703 in Miami Beach,"$6,700,000","2,615","$2,562",2024,Nelson Gonzalez BHHS EWM Realty,Jordan Karp Jordan Karp LLC
3,2025-08-05,BELLINI CONDO,10225 Collins Ave #901 in Bal Harbour,"$4,700,000","3,820","$1,230",2004,Jill Hertzberg Coldwell Banker Realty,Raquel Sragowicz Citadel Realty
4,2025-08-07,PRIVE CONDO,5000 Island Estates Dr #1208 in Aventura,"$4,125,000","4,119","$1,001",2018,Richard Goihman Douglas Elliman,Salesperson Non Member Non-MLS Member
5,2025-08-06,NORTH CARILLON BEACH COND,6899 Collins Ave #2804 in Miami Beach,"$3,530,000","2,150","$1,642",2008,Eileen Nexer Douglas Elliman Levente Kovary Do...,Levente Kovary Douglas Elliman
6,2025-08-07,Roney Palace,2301 Collins Ave #1407 in Miami Beach,"$2,400,000","1,230","$1,951",1926,Erin Argiropoulos RE/MAX Preferred Rachael Bar...,Sophia Albrecht Thine Agency Inc
7,2025-08-05,The Sands of Key Biscayne,605 Ocean Dr Unit 4L in Key Biscayne,"$2,120,000","1,772","$1,196",1969,Rene Iglesias One Sotheby's International Real...,Julian Osuna Ospina Keller Williams Capital Re...
8,2025-08-05,EPIC WEST CONDO,200 Biscayne Boulevard Way #4702 in Miami,"$1,750,000","2,457",$712,2008,Iria Zarraga Fortune Christie's Intl R.E.,Erica Tarafa The Keyes Company
9,2025-08-05,CAPE FLA CLUB CONDO CLUST,288 Sea View Dr Unit C2 in Key Biscayne,"$1,665,000","1,800",$925,1970,Patricia Romano Coldwell Banker Realty Patrici...,"Heide Arguelles Keys Gate Realty, Inc."


In [78]:
csv_date_string = today.strftime("%m_%d_%Y")

In [79]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [80]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')